In [ ]:
import requests
import datetime

def download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_file):
    """
    Downloads a GRIB2 file from NOAA for a specified area and forecast hour.

    Parameters:
        min_lat (float): Minimum latitude of the area.
        max_lat (float): Maximum latitude of the area.
        min_lon (float): Minimum longitude of the area.
        max_lon (float): Maximum longitude of the area.
        forecast_hour (int): Forecast hour (e.g., 240 for 10-day forecast).
        output_file (str): Path to save the downloaded GRIB2 file.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.timezone.utc)
    run_date = now.strftime("%Y%m%d")
    run_hour = "00"  # Default to 00 UTC; change as needed

    # Construct the URL for the GRIB2 file
    file_name = f"gfs.t{run_hour}z.pgrb2b.0p25.f{forecast_hour:03d}"
    base_url = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/"
    file_url = f"{base_url}gfs.{run_date}/{run_hour}/atmos/{file_name}"

    try:
        # Request to download the GRIB2 file
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Check for HTTP errors

        # Save the file locally
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"GRIB2 file downloaded successfully: {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading GRIB2 file: {e}")
        print(f"URL attempted: {file_url}")


In [4]:
import datetime

def get_latest_grib_time():
    """
    Calculates the latest GRIB file base time in the format required for downloading.

    Returns:
        str: The latest base time in 'YYYYMMDD/HH' format.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.UTC)
    print(f"Current UTC time: {now.strftime('%c')}")

    # GFS model updates every 6 hours (00, 06, 12, 18 UTC)
    # Find the most recent model run hour
    recent_run_hour = (now.hour // 6) * 6
    recent_run_time = now.replace(hour=recent_run_hour, minute=0, second=0, microsecond=0)

    # If the current time is before the most recent run's availability, go back one cycle
    # GFS files are typically available ~4 hours after the run starts
    if now < recent_run_time + datetime.timedelta(hours=4):
        recent_run_time -= datetime.timedelta(hours=6)

    # Format as 'YYYYMMDD/HH'
    return recent_run_time.strftime("%Y%m%d"), recent_run_time.strftime("%H")

# Example usage
latest_grib_time = get_latest_grib_time()
print(f"Latest GRIB time: {latest_grib_time}")

Current UTC time: Tue Dec 10 02:49:15 2024
Latest GRIB time: ('20241209', '18')


In [108]:
# Example usage
##################
# Define the geographic bounds (Note: bounds are ignored for GRIB2b files)
bounds = {'lat': (32.0, 34.5),
          'lng': (-122.0, -116.0) }
##################
# Waypoints for the Channel islands 500
waypoints = [(34.40161, -119.69426),  #SBYC (start)
             (34.08265, -119.98985),  #Santa Cruz channel (north)
             (33.95741, -119.91079),  #Santa Cruz channel (south)
             (32.96459, -121.34031),  #Western Mark
             (31.64010, -119.82025),  #Southern Mark
             (32.75008, -117.49959),  #Finish
            ]
##################
# find the time of the latest forecast calculation
(FCdate, FCtime) = get_latest_grib_time()
print(FCdate, FCtime)
##################
# at t=0, find the wind a the first waypoint
simulation_time=0
grib_file = download_nomads_gfs_forecast_file(FCdate, FCtime, bounds['lat'], bounds['lng'], simulation_time)
for ndx in range(len(waypoints)):
    (wind_speed, wind_direction) = get_wind_at_location(grib_file, waypoints[ndx][0], waypoints[ndx][1], simulation_time)
    print(waypoints[ndx][0], waypoints[ndx][1], wind_speed, wind_direction)


Current UTC time: Tue Dec 10 06:40:41 2024
20241210 00
238.0 244.0
https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241210%2F00%2Fatmos&file=gfs.t00z.pgrb2.0p25.f000&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238.0&rightlon=244.0&bottomlat=32.0
gfs.20241210-00-gfs.t00z.pgrb2.0p25.f000
starting download
GRIB2 file downloaded successfully: gfs.20241210-00-gfs.t00z.pgrb2.0p25.f000
34.40161 -119.69426 12.45485890920352 344.85712031298647
34.08265 -119.98985 11.693220613472459 340.8837805908953
33.95741 -119.91079 9.57860309845321 356.31701035230924
32.96459 -121.34031 5.666072030108454 22.500002587192625
31.6401 -119.82025 9.614480912585584 3.971105184225962
32.75008 -117.49959 7.095144947981734 353.6222324378846


In [120]:
def simulate_shortest_path(lat_start,lng_start, lat_end, lng_end):
    (FCdate, FCtime) = get_latest_grib_time()
    print('starting time:',FCdate, FCtime)
    simulation_time = 0
    lat = lat_start
    lng = lng_start
    last_dist_togo = haversine_distance(lat,lng,lat_end,lng_end);
    #####
    for _ in range(0,10): # 10 steps
    
        grib_file = download_nomads_gfs_forecast_file(FCdate, FCtime, bounds['lat'], bounds['lng'], simulation_time)
        (tws, twd) = get_wind_at_location(grib_file, lat, lng, simulation_time)
        print(f"{simulation_time}: ({lat},{lng}) tws={tws} twd={twd}")
        polars = polar_rhiannon(tws)
        # find all possible angles
        best_dist_togo = None
        best_nav_args = {}
        for (angle,boat_speed) in polars:
            for delta_angle in (angle, -1*angle):
                boat_mag = (twd+delta_angle)%360
                (dlat,dlng) = calculate_destination_latlng(lat,lng,boat_speed,boat_mag,1) # hour
                dist_to_dest = haversine_distance(dlat,dlng,lat_end,lng_end)
                #print(f" twa={angle} mag={boat_mag:.1f} dest=({dlat:.1f},{dlng:.1f}) dist_togo={dist_to_dest:.1f}")
                if best_dist_togo is None or dist_to_dest < best_dist_togo:
                    best_dist_togo = dist_to_dest
                    best_nav_args['twa']=angle
                    best_nav_args['mag']=boat_mag
                    best_nav_args['lat']=dlat
                    best_nav_args['lng']=dlng
                    best_nav_args['dist_togo']=dist_to_dest
        print('best_nav',best_nav_args)
        if best_dist_togo < last_dist_togo:
            simulation_time+=1
            lat=best_nav_args['lat']
            lng=best_nav_args['lng']
            last_dist_togo = best_dist_togo
        else:
            break
            
                


simulate_shortest_path(waypoints[0][0], waypoints[0][1], waypoints[1][0], waypoints[1][1])

Current UTC time: Tue Dec 10 07:05:38 2024
starting time: 20241210 00
238.0 244.0
https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241210%2F00%2Fatmos&file=gfs.t00z.pgrb2.0p25.f000&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238.0&rightlon=244.0&bottomlat=32.0
gfs.20241210-00-gfs.t00z.pgrb2.0p25.f000
starting download
GRIB2 file downloaded successfully: gfs.20241210-00-gfs.t00z.pgrb2.0p25.f000
0: (34.40161,-119.69426) tws=12.45485890920352 twd=344.85712031298647
best_nav {'twa': 120, 'mag': 224.85712031298647, 'lat': 34.30852364102082, 'lng': -119.8063311272444, 'dist_togo': 16.339379086307424}
238.0 244.0
https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241210%2F00%2Fatmos&file=gfs.t00z.pgrb2.0p25.f001&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238.0&rightlon=244.0&bottomlat=32.0
gfs.20241210-00-gfs.t00z.pgrb2.0p25.f001
starting download
GRIB2 file downloaded succes

In [121]:
# lat = 34.40161 
# lng = -119.69426 + 360
# lat_end = waypoints[1][0]
# lng_end = waypoints[1][1]
# tws = 11
# twd = 344.8
# polars = polar_rhiannon(tws)
# for (angle,boat_speed) in polars:
#     for delta_angle in (angle, -1*angle):
#         boat_mag = (twd+delta_angle)%360
#         (dlat,dlng) = calculate_destination_latlng(lat,lng,boat_speed,boat_mag,1) # hour
#         dist_to_dest = haversine_distance(dlat,dlng,lat_end,lng_end)
#         print(f" twa={angle} mag={boat_mag:.1f} dest=({dlat:.1f},{dlng:.1f}) dist_to_dest={dist_to_dest:.1f}")
    
    

In [6]:
# docs here: https://nomads.ncep.noaa.gov/gribfilter.php?ds=gfs_0p25_1hr
import requests

def download_nomads_gfs_forecast_file(grib_date, grib_time, lat_bounds, lng_bounds, simulation_time):
    min_lng = lng_bounds[0] if lng_bounds[0]>0 else lng_bounds[0]+360
    max_lng = lng_bounds[1] if lng_bounds[1]>0 else lng_bounds[1]+360
    print(min_lng,max_lng)
    min_lat = lat_bounds[0]
    max_lat = lat_bounds[1]
    #https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241203%2F12%2Fatmos&file=gfs.t12z.pgrb2.0p25.f240&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238&rightlon=244&bottomlat=32
    url = f"https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.{grib_date}%2F{grib_time}%2Fatmos&file=gfs.t{grib_time}z.pgrb2.0p25.f{simulation_time:03}&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat={max_lat}&leftlon={min_lng}&rightlon={max_lng}&bottomlat={min_lat}"
    print(url)
    output_file = f"gfs.{grib_date}-{grib_time}-gfs.t{grib_time}z.pgrb2.0p25.f{simulation_time:03}"
    print(output_file)
    
    try:
        # Make the request to download the GRIB2 file
        print(f"starting download")
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP errors
        
        # Save the file locally
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"GRIB2 file downloaded successfully: {output_file}")
        return output_file
    except requests.exceptions.RequestException as e:
        print(f"Error downloading GRIB2 file: {e}")
        raise e
    

In [7]:
import math
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the distance between two points on the Earth in nautical miles.
    
    Parameters:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
    
    Returns:
        float: Distance in nautical miles.
    """
    # Radius of Earth in nautical miles
    R = 3440.065
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Distance in nautical miles
    distance = R * c
    return distance

# Example usage
lat1, lon1 = 34.0, -120.0  # Point 1 (latitude, longitude)
lat2, lon2 = 33.0, -119.0  # Point 2 (latitude, longitude)
print(f"Distance: {haversine_distance(lat1, lon1, lat2, lon2):.2f} nautical miles")


Distance: 78.18 nautical miles


In [60]:
def mps_to_knots(speed_mps):
    """
    Convert speed from meters per second (m/s) to knots.

    Parameters:
        speed_mps (float): Speed in meters per second.

    Returns:
        float: Speed in knots.
    """
    # 1 knot = 1.9438444924406 m/s
    knots_per_mps = 1.9438444924406
    return speed_mps * knots_per_mps

# Example usage
speed_mps = 10  # Example speed in m/s
speed_knots = mps_to_knots(speed_mps)
print(f"{speed_mps} m/s is equal to {speed_knots:.2f} knots")


10 m/s is equal to 19.44 knots


In [76]:
import pygrib
import numpy as np
from math import atan2, degrees, sqrt

def get_wind_at_location(grib_file, mylat, mylng, simulation_time, verbose=False):
    """
    Extracts wind speed and direction at a given GPS coordinate and time from a GRIB2 file.

    Parameters:
        grib_file (str): Path to the GRIB2 file.
        lat (float): Latitude of the location.
        lng (float): Longitude of the location.
        simulation_time (int): Forecast hour to extract data for.
        verbose (boolean): print debugging info
    Returns:
        dict: A dictionary with wind speed (m/s) and direction (degrees).
    """
    try:
        # Open the GRIB2 file
        with pygrib.open(grib_file) as grbs:
            # Filter messages for U and V wind components at 10m above ground and the given forecast time
            # Extract the data and corresponding lat/lon grid
            u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=simulation_time)[0].values
            v_msg = grbs.select(name="10 metre V wind component", level=10, forecastTime=simulation_time)[0].values
            lats, lngs = grbs[1].latlons()


            # Interpolate U and V components at the exact point
            for lat_ndx in range(0, lats.shape[0]):
                #print(lat_ndx, lats[lat_ndx][0])
                if lats[lat_ndx][0] > mylat: break
            for lng_ndx in range(0, lngs.shape[1]):
                #print(lng_ndx, lngs[0][lng_ndx])
                if lngs[0][lng_ndx] > mylng: break
            ndx_n = lat_ndx
            ndx_s = lat_ndx-1
            ndx_w = lng_ndx
            ndx_e = lng_ndx-1
            d_nw = haversine_distance(mylat,mylng, lats[ndx_n][0], lngs[0][ndx_w])
            d_ne = haversine_distance(mylat,mylng, lats[ndx_n][0], lngs[0][ndx_e])
            d_sw = haversine_distance(mylat,mylng, lats[ndx_s][0], lngs[0][ndx_w])
            d_se = haversine_distance(mylat,mylng, lats[ndx_s][0], lngs[0][ndx_e])
            w_nw = d_nw / (d_nw+d_ne+d_sw+d_se)
            w_ne = d_ne / (d_nw+d_ne+d_sw+d_se)
            w_sw = d_sw / (d_nw+d_ne+d_sw+d_se)
            w_se = d_se / (d_nw+d_ne+d_sw+d_se)
            if verbose:
                print(f"NW: ({lats[ndx_n][0]},{lngs[0][ndx_w]})  d={d_nw} w={w_nw} u={u_msg[ndx_n][ndx_w]} v={v_msg[ndx_n][ndx_w]}")
                print(f"NE: ({lats[ndx_n][0]},{lngs[0][ndx_e]})  d={d_ne} w={w_ne} u={u_msg[ndx_n][ndx_e]} v={v_msg[ndx_n][ndx_e]}")
                print(f"SW: ({lats[ndx_s][0]},{lngs[0][ndx_w]})  d={d_sw} w={w_sw} u={u_msg[ndx_s][ndx_w]} v={v_msg[ndx_s][ndx_w]}")
                print(f"SE: ({lats[ndx_s][0]},{lngs[0][ndx_e]})  d={d_se} w={w_se} u={u_msg[ndx_s][ndx_e]} v={v_msg[ndx_s][ndx_e]}")
            
            v_wind = w_nw * v_msg[ndx_n][ndx_w] \
                   + w_ne * v_msg[ndx_n][ndx_e] \
                   + w_sw * v_msg[ndx_s][ndx_w] \
                   + w_se * v_msg[ndx_s][ndx_e]
            u_wind = w_nw * u_msg[ndx_n][ndx_w] \
                   + w_ne * u_msg[ndx_n][ndx_e] \
                   + w_sw * u_msg[ndx_s][ndx_w] \
                   + w_se * u_msg[ndx_s][ndx_e]            
            
            # Calculate wind speed (m/s->knots) and direction (degrees)
            wind_speed = mps_to_knots( sqrt(u_wind**2 + v_wind**2) )
            wind_direction = (270 - degrees(atan2(v_wind, u_wind))) % 360
            if verbose:
                print(f"Wind: u={u_wind} v={v_wind}")
                print(f"wind_speed {wind_speed}, wind_direction {wind_direction}")
            return (wind_speed, wind_direction)

    except Exception as e:
        print(f"Error processing GRIB2 file: {e}")
        return None




In [96]:

def polar_rhiannon(wind_speed):
    """
    Return list of (angle, boat_speed) pairs for a given input wind_speed
    """
    angles = [52,60,75,90,110,120,135,150,165,180]
    
    tws = [6 ,	8 ,	10 ,	12 ,	16 ,	20 ,	24] #    'True wind speed':,
    
    data = {
    'Optimum Beat':[(48.2,3.87),(46.9,4.85),(45.8,5.44),(44.8,6.13),(43.4,6.6),(41.9,6.77),(41.8,6.87)],
    'Optimum Run':[(132.2,4.68),(139.1,5.33),(149.5,5.61),(151.4,6.32),(163.7,7.09),(170.7,7.73),(172.3,8.31)],
    '52-degrees':[	4.17,	5.3,	6.09,	6.72,	7.26,	7.48,	7.57],
    '60-degrees':[	4.68,	5.79,	6.67,	7.17,	7.64,	7.85,	7.97],
    '75-degrees':[	5.63,	6.85,	7.5,	7.83,	8.23,	8.47,	8.61],
    '90-degrees':[	6.04,	7.21,	7.81,	8.17,	8.6,	8.89,	9.1],
    '110-degrees':[	5.95,	7.14,	7.73,	8.15,	8.81,	9.29,	9.57],
    '120-degrees':[	5.62,	6.77,	7.45,	7.88,	8.57,	9.19,	9.75],
    '135-degrees':[	4.44,	5.66,	6.59,	7.28,	8.1,	8.74,	9.36],
    '150-degrees':[	3.45,	4.57,	5.58,	6.4,	7.59,	8.29,	8.88],
    '165-degrees':[	2.9,	3.88,	4.81,	5.66,	7.04,	7.87,	8.47],
    '180-degrees':[	2.61,	3.52,	4.39,	5.2,	6.59,	7.56,	8.18],
    }
    for ndx in range(len(tws)):
        if tws[ndx] > wind_speed: break
    ndx-=1 #ndx of speed less than wind speed
    print('ndx',ndx)

    polars = []
    # beat
    if ndx==-1:
        ta = data['Optimum Beat'][0][0]
        ts = data['Optimum Beat'][0][1] * (wind_speed/tws[0])
        polars.append( (ta,ts) )
    else:
        polars.append( data['Optimum Beat'][ndx] )
    # run through angles
    for angle in angles:
        if ndx==-1:
            ts = data[f"{angle}-degrees"][0] * (wind_speed/tws[0])
            polars.append( (angle,ts) )
        else:
            polars.append( (angle, data[f"{angle}-degrees"][ndx]) )
    # run
    if ndx==-1:
        ta = data['Optimum Run'][0][0]
        ts = data['Optimum Run'][0][1] * (wind_speed/tws[0])
        polars.append( (ta,ts) )
    else:
        polars.append( data['Optimum Run'][ndx] )

    # return pairs of (angle, ba
    return polars


polar_rhiannon(3)

[(48.2, 1.935),
 (52, 2.085),
 (60, 2.34),
 (75, 2.815),
 (90, 3.02),
 (110, 2.975),
 (120, 2.81),
 (135, 2.22),
 (150, 1.725),
 (165, 1.45),
 (180, 1.305),
 (132.2, 2.34)]

In [106]:
import math

def calculate_destination_latlng(lat, lon, speed_knots, direction, travel_time_hours):
    """
    Calculate the destination latitude and longitude.

    Parameters:
        lat (float): Starting latitude in decimal degrees.
        lon (float): Starting longitude in decimal degrees.
        speed_knots (float): Speed in knots.
        direction (float): Direction in degrees (0° is North, 90° is East).
        travel_time_hours (float): Travel time in hours.

    Returns:
        tuple: (destination_lat, destination_lon) in decimal degrees.
    """
    # Convert speed from knots to nautical miles per hour (1 knot = 1 nautical mile per hour)
    distance_nm = speed_knots * travel_time_hours
    
    # Earth's radius in nautical miles
    R = 3440.065
    
    # Convert inputs to radians
    lat = math.radians(lat)
    lon = math.radians(lon)
    direction = math.radians(direction)
    
    # Calculate the destination latitude
    destination_lat = math.asin(
        math.sin(lat) * math.cos(distance_nm / R) +
        math.cos(lat) * math.sin(distance_nm / R) * math.cos(direction)
    )
    
    # Calculate the destination longitude
    destination_lon = lon + math.atan2(
        math.sin(direction) * math.sin(distance_nm / R) * math.cos(lat),
        math.cos(distance_nm / R) - math.sin(lat) * math.sin(destination_lat)
    )
    
    # Convert the results back to degrees
    destination_lat = math.degrees(destination_lat)
    destination_lon = math.degrees(destination_lon)
    
    # Normalize longitude to be within the range [-180, 180]
    destination_lon = (destination_lon + 180) % 360 - 180
    
    return destination_lat, destination_lon

# Example usage
start_lat = 34.0       # Starting latitude
start_lon = -120.0     # Starting longitude
speed_knots = 10       # Speed in knots
direction = 45         # Direction in degrees (NE)
travel_time_hours = 2  # Travel time in hours

dest_lat, dest_lon = calculate_destination_latlng(start_lat, start_lon, speed_knots, direction, travel_time_hours)
print(f"Destination: Latitude {dest_lat:.6f}, Longitude {dest_lon:.6f}")


Destination: Latitude 34.235215, Longitude -119.715092


In [12]:
# with pygrib.open(grib_file) as grbs:
#     # Filter messages for U and V wind components at 10m above ground and the given forecast time
#     u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=simulation_time)[0]
#     v_msg = grbs.select(name="10 metre V wind component", level=10, forecastTime=simulation_time)[0]


In [9]:
# gps_lat = 33.0  # Example latitude
# gps_lon = -118.0 + 360  # Example longitude
# get_wind_at_location("gfs.20241204-18-gfs.t18z.pgrb2.0p25.f000A", gps_lat, gps_lon, 240)

In [73]:
# gps_lat = 33.0  # Example latitude
# gps_lon = -118.0 + 360  # Example longitude
# grbs = pygrib.open(grib_file)
# for grb in grbs:
#     print(grb)  # Prints metadata about each message in the file
#     print(f"Forecast time: {grb.forecastTime} hours, Parameter: {grb.name}")
# u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=simulation_time)[0].values
# v_msg = grbs.select(name="10 metre V wind component", level=10, forecastTime=simulation_time)[0].values
# #print(u_msg)
# #print(u_msg.data())
# x = u_msg
# x.shape

In [72]:
#lats, lngs = grb.latlons()
#lats.shape
#print(lats)
#print()
#print(lngs)

In [71]:
# mylat = 33.977722990833655
# mylng = -118.45751151468153 + 360
# print(mylat,mylng)

# for lat_ndx in range(0, lats.shape[0]):
#     #print(lat_ndx, lats[lat_ndx][0])
#     if lats[lat_ndx][0] > mylat: break
# for lng_ndx in range(0, lngs.shape[1]):
#     #print(lng_ndx, lngs[0][lng_ndx])
#     if lngs[0][lng_ndx] > mylng: break
# print('Bounds')
# ndx_n = lat_ndx
# ndx_s = lat_ndx-1
# ndx_w = lng_ndx
# ndx_e = lng_ndx-1
# d_nw = haversine_distance(mylat,mylng, lats[ndx_n][0], lngs[0][ndx_w])
# d_ne = haversine_distance(mylat,mylng, lats[ndx_n][0], lngs[0][ndx_e])
# d_sw = haversine_distance(mylat,mylng, lats[ndx_s][0], lngs[0][ndx_w])
# d_se = haversine_distance(mylat,mylng, lats[ndx_s][0], lngs[0][ndx_e])
# w_nw = d_nw / (d_nw+d_ne+d_sw+d_se)
# w_ne = d_ne / (d_nw+d_ne+d_sw+d_se)
# w_sw = d_sw / (d_nw+d_ne+d_sw+d_se)
# w_se = d_se / (d_nw+d_ne+d_sw+d_se)
# print(f"NW: ({lats[ndx_n][0]},{lngs[0][ndx_w]})  d={d_nw} w={w_nw} u={u_msg[ndx_n][ndx_w]} v={v_msg[ndx_n][ndx_w]}")
# print(f"NE: ({lats[ndx_n][0]},{lngs[0][ndx_e]})  d={d_ne} w={w_ne} u={u_msg[ndx_n][ndx_e]} v={v_msg[ndx_n][ndx_e]}")
# print(f"SW: ({lats[ndx_s][0]},{lngs[0][ndx_w]})  d={d_sw} w={w_sw} u={u_msg[ndx_s][ndx_w]} v={v_msg[ndx_s][ndx_w]}")
# print(f"SE: ({lats[ndx_s][0]},{lngs[0][ndx_e]})  d={d_se} w={w_se} u={u_msg[ndx_s][ndx_e]} v={v_msg[ndx_s][ndx_e]}")

# v_wind = w_nw * v_msg[ndx_n][ndx_w] \
#        + w_ne * v_msg[ndx_n][ndx_e] \
#        + w_sw * v_msg[ndx_s][ndx_w] \
#        + w_se * v_msg[ndx_s][ndx_e]
# u_wind = w_nw * u_msg[ndx_n][ndx_w] \
#        + w_ne * u_msg[ndx_n][ndx_e] \
#        + w_sw * u_msg[ndx_s][ndx_w] \
#        + w_se * u_msg[ndx_s][ndx_e]

# print(f"wind: u={u_wind} v={v_wind}")


# # Calculate wind speed (m/s->knots) and direction (degrees)
# wind_speed = mps_to_knots( sqrt(u_wind**2 + v_wind**2) )
# wind_direction = (270 - degrees(atan2(v_wind, u_wind))) % 360

#{"wind_speed": wind_speed, "wind_direction": wind_direction}



In [ ]:
# # Example usage
# # Path to the GRIB2 file
# grib_file_path = "gfs_10day_forecast.grib2"

# # Specify the GPS coordinates and forecast time
# gps_lat = 33.0  # Example latitude
# gps_lon = -120.0  # Example longitude
# forecast_hour = 0  

# # Get wind speed and direction
# result = get_wind_at_location(grib_file_path, gps_lat, gps_lon, forecast_hour)
# if result:
#     print(f"Wind speed: {result['wind_speed']} m/s")
#     print(f"Wind direction: {result['wind_direction']} degrees")

In [70]:
# grib_file_path = "gfs_10day_forecast.grib2"
# with pygrib.open(grib_file_path) as grbs:
#     for grb in grbs:
#         print(grb)  # Prints metadata about each message in the file